In [2]:
import requests
from bs4 import BeautifulSoup, element
import numpy as np 
import pandas as pd 
from datetime import datetime
import os
import io
import re

In [3]:
data = pd.read_csv('mypaper.csv')

In [4]:
data

,Unnamed: 0,date,domain,url,text
0,0,2019-09-30,www.mypaperonline.com,https://www.mypaperonline.com/category/the-ran...,The Randolph News Archive My Paper Online Hom...
1,1,2019-10-01,www.mypaperonline.com,https://www.mypaperonline.com/category/local-n...,Local News Archive My Paper Online Home Discl...
2,2,2019-10-01,www.mypaperonline.com,https://www.mypaperonline.com/local-art-exhibi...,Local Art Exhibit And Sale At CCM My Paper On...
3,3,2019-10-01,www.mypaperonline.com,https://www.mypaperonline.com/wp-json/oembed/1...,{version:1.0provider_name:My Paper Onlineprovi...
4,4,2019-10-12,www.mypaperonline.com,https://www.mypaperonline.com/category/the-hac...,The Hackettstown News Archive My Paper Online...
...,...,...,...,...,...
2556,4332,2019-10-12,www.mypaperonline.com,https://www.mypaperonline.com/hauntings-around...,Hauntings Around the Area for Halloween My Pa...
2557,4333,2019-10-12,www.mypaperonline.com,https://www.mypaperonline.com/west-orange-resi...,West Orange Resident Centenary Student Studies...
2558,4334,2019-10-12,www.mypaperonline.com,https://www.mypaperonline.com/18-year-old-west...,18-Year-Old West Orange Teen Battles Cancer an...
2559,4335,2019-10-12,www.mypaperonline.com,https://www.mypaperonline.com/millburn-residen...,Millburn Resident Wins Award for Activism My...


In [76]:

def get_html(collection_id, name, data):

    if not os.path.isdir(name):
        os.mkdir(name)

    ai_url = "https://wayback.archive-it.org"

    date_list = pd.to_datetime(data['date'].unique())
    urls = data['url'].unique()

    ctr = 0

    for url in urls:

        ctr = ctr + 1

        home = ai_url + '/' + str(collection_id) + '/*/' + url

        page = requests.get(home)
        soup = BeautifulSoup(page.content, "lxml")
        alist = soup.find_all("a")
        alist = [a for a in alist if 'onclick' in a.attrs]
        dates = [datetime.strptime(a.text, '%b %d, %Y') for a in alist]

        for a in alist:

            if 'onclick' in a.attrs and datetime.strptime(a.text, '%b %d, %Y') in date_list:
                wp = requests.get('https:' + a.attrs['href'])
                path = name + '/' + datetime.strptime(a.text, '%b %d, %Y').strftime('%Y%m%d') + '_' + ''.join(re.findall('(\d+|[a-zA-Z]+|-|\.)', url.split('/')[-1]))
                print(path)
                if not os.path.isfile(path):
                    print('write')
                    f = open(path, 'wb')
                    f.write(wp.content)
                    f.close()




In [79]:
get_html(12706, 'mypaper_full', data)

mypaper_full/20191008_embedurlhttps3A2F2Fwww.mypaperonline.com2Fthe-cartells.html
mypaper_full/20191008_embedurlhttps3A2F2Fwww.mypaperonline.com2Fthe-cartells.html
mypaper_full/20191012_embedurlhttps3A2F2Fwww.mypaperonline.com2Fthe-cartells.html
write
mypaper_full/20191012_embedurlhttps3A2F2Fwww.mypaperonline.com2Fthe-cartells.html
mypaper_full/20191012_embedurlhttps3A2F2Fwww.mypaperonline.com2Fthe-cartells.html
mypaper_full/20191012_embedurlhttps3A2F2Fwww.mypaperonline.com2Fthe-cartells.html
mypaper_full/20191020_embedurlhttps3A2F2Fwww.mypaperonline.com2Fthe-cartells.html
write
mypaper_full/20191020_embedurlhttps3A2F2Fwww.mypaperonline.com2Fthe-cartells.html
mypaper_full/20191020_embedurlhttps3A2F2Fwww.mypaperonline.com2Fthe-cartells.html
mypaper_full/20191020_embedurlhttps3A2F2Fwww.mypaperonline.com2Fthe-cartells.html
mypaper_full/20191024_embedurlhttps3A2F2Fwww.mypaperonline.com2Fthe-cartells.html
write
mypaper_full/20191024_embedurlhttps3A2F2Fwww.mypaperonline.com2Fthe-cartells.htm

In [5]:

data['morristown'] = None

for index, row in data.iterrows():

    path = 'mypaper_full/' + ''.join(re.findall('\d+', row.date)) + '_' + ''.join(re.findall('(\d+|[a-zA-Z]+|-|\.)', row.url.split('/')[-1]))

    try:
        f = open(path, 'rb')
        html = f.read()
    except:
        raise Exception('nope')

    try:
        soup = BeautifulSoup(html, "lxml")
        article = soup.find('article').attrs['class']
    except:
        article = []
        data.at[index, 'morristown'] = 0
    
    if 'category-the-morristown-news' in article:
        data.at[index, 'morristown'] = 1
    else:
        data.at[index, 'morristown'] = 0



In [81]:
display(data)

,Unnamed: 0,date,domain,url,text,morristown
0,0,2019-09-30,www.mypaperonline.com,https://www.mypaperonline.com/category/the-ran...,The Randolph News Archive My Paper Online Hom...,0
1,1,2019-10-01,www.mypaperonline.com,https://www.mypaperonline.com/category/local-n...,Local News Archive My Paper Online Home Discl...,1
2,2,2019-10-01,www.mypaperonline.com,https://www.mypaperonline.com/local-art-exhibi...,Local Art Exhibit And Sale At CCM My Paper On...,0
3,3,2019-10-01,www.mypaperonline.com,https://www.mypaperonline.com/wp-json/oembed/1...,{version:1.0provider_name:My Paper Onlineprovi...,0
4,4,2019-10-12,www.mypaperonline.com,https://www.mypaperonline.com/category/the-hac...,The Hackettstown News Archive My Paper Online...,0
...,...,...,...,...,...,...
2556,4332,2019-10-12,www.mypaperonline.com,https://www.mypaperonline.com/hauntings-around...,Hauntings Around the Area for Halloween My Pa...,1
2557,4333,2019-10-12,www.mypaperonline.com,https://www.mypaperonline.com/west-orange-resi...,West Orange Resident Centenary Student Studies...,0
2558,4334,2019-10-12,www.mypaperonline.com,https://www.mypaperonline.com/18-year-old-west...,18-Year-Old West Orange Teen Battles Cancer an...,0
2559,4335,2019-10-12,www.mypaperonline.com,https://www.mypaperonline.com/millburn-residen...,Millburn Resident Wins Award for Activism My...,0


In [83]:
data.to_csv('labeled.csv')

In [4]:
labeled = pd.read_csv('labeled.csv')

In [7]:
morristown_news = labeled[(labeled.url.str.contains('https://www.mypaperonline.com/category/the-morristown-news') == True) | (labeled.morristown == 1)]

In [8]:
morristown_news.to_csv('morristown.csv')